In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

from CRUDpythonMod import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "aacpwd"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#     html.Div(id='hidden-div', style={'display':'block', 'background-color': 'green'}),
    
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
        
    html.H4(html.B("Alex Metzger's Interactive Datatable")),
    
    html.Hr(),
    html.Div(className='top-row', style={'display': 'block', 'width': '50%'},
            children=[
            html.P("Select Rescue Type..."),
                
            dcc.Dropdown(
                id='dropdown',
                options=[{'label': 'Reset', 'value': "none"},
                     {'label': 'Water Rescue', 'value': "Water"},
                     {'label': 'Mountain/Wilderness Rescue', 'value': "Mountain"},
                     {'label': 'Disaster Rescue/Individual Tracking', 'value': "Disaster"}
                    ],
                value='none',
                clearable = True
            )]
             
    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        page_size=10,
        data=df.to_dict('records'),
        filter_action = 'native',
        sort_action = 'native',
        row_selectable = 'single'
    ),
    html.Br(),
     html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex', 'width': '100%'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%', 'height': '500px'}

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%', 'height': '500px'}
            )
        ])
])

def select_query(rescue_type):
    '''This function is used to select the query to search the mongo database for
       based on the dropdown menu that is selected for the rescue_type filter'''
    
    if rescue_type == 'none':
    # Search all animals if no rescue type is selected from dropdown
        query = {}

    elif rescue_type == 'Water':
    # If 'Water Rescue' is selected from dropdown, filter all breeds that are either:
    #      Labrador Retriever Mix
    #      Chesa Bay Retr Mix
    #      Newfoundlound
    # AND filter gender
    #      Intact Female
    # AND filter age
    #     26 < age_in_weeks < 156
    
        query = {"$or": [{"breed": "Labrador Retriever Mix"},
                         {"breed": {"$regex":".*Chesa Bay Retr.*"}},
                         {"breed": {"$regex":".*Newfoundland.*"}}
                        ],
                 "$and": [{"sex_upon_outcome": "Intact Female"}, 
                          {"$and": [{"age_upon_outcome_in_weeks": {"$gt": 26}},
                                    {"age_upon_outcome_in_weeks": {"$lt": 156}}]
                          }
                         ]
                }
        
    elif rescue_type == 'Mountain':
    # If 'Mountain/Wilderness Rescue' is selected from dropdown, filter all breeds that are either:
    #      German Shepherd
    #      Alaskan Malamute
    #      Old English Sheepdog
    #      Siberian Husky
    #      Rottweiler
    # AND filter gender
    #      Intact Male
    # AND filter age
    #     26 < age_in_weeks < 156
    
        query = {"$or": [{"breed": {"$regex":".*German Shepherd.*"}},
                         {"breed": {"$regex":".*Alaskan Malamute.*"}},
                         {"breed": {"$regex":".*Old English Sheepdog.*"}},
                         {"breed": {"$regex":".*Siberian Husky.*"}},
                         {"breed": {"$regex":".*Rottweiler.*"}}
                        ],
                 "$and": [{"sex_upon_outcome": "Intact Male"}, 
                          {"$and": [{"age_upon_outcome_in_weeks": {"$gt": 26}},
                                    {"age_upon_outcome_in_weeks": {"$lt": 156}}]
                          }
                         ]
                }
                 
    elif rescue_type == 'Disaster':
    # If 'Disaster Rescue/' is selected from dropdown, filter all breeds that are either:
    #      Doberman Pinsch
    #      German Shepherd
    #      Golden Retriever
    #      Bloodhound
    #      Rottweiler
    # AND filter gender
    #      Intact Male
    # AND filter age
    #     20 < age_in_weeks < 300
    
        query = {"$or": [{"breed": {"$regex":".*Doberman Pinsch.*"}},
                         {"breed": {"$regex":".*German Shepherd.*"}},
                         {"breed": {"$regex":".*Golden Retriever.*"}},
                         {"breed": {"$regex":".*Bloodhound.*"}},
                         {"breed": {"$regex":".*Rottweiler.*"}}
                        ],
                 "$and": [{"sex_upon_outcome": "Intact Male"}, 
                          {"$and": [{"age_upon_outcome_in_weeks": {"$gt": 20}},
                                    {"age_upon_outcome_in_weeks": {"$lt": 300}}]
                          }
                         ]
                }  
    
    return query
#############################################
# Interaction Between Components / Controller
#############################################

# This callback will filter the table by rescue type
@app.callback(Output('datatable-id', "data"),
              [Input('dropdown', 'value')])

def update_dashboard(dropdown_val):
    # Select query based on dropdown value selected
    rescue_query = select_query(dropdown_val)
    # Make dataframe out of the query
    df = pd.DataFrame.from_records(shelter.read(rescue_query))
    
    return df.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback is responsible for updating the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('dropdown', 'value')])
     
def update_graphs(dropdown_val):
     # Select query based on dropdown value selected
    rescue_query = select_query(dropdown_val)
    # Make dataframe based on query
    dff = pd.DataFrame.from_records(shelter.read(rescue_query))
    
    
    # Get totals for each breed
    dff_breed_totals = dff['breed'].value_counts()
    # Create new dataframe with the breed_totals series
    df_result = pd.DataFrame(dff_breed_totals)
    df_result = df_result.reset_index()  
    df_result.columns = ['Breed', 'Total']
    
    # List only top 5 breeds in labels if no rescue filter applied, else
    # list top 8. The rest will be labeled 'other'
    if dropdown_val == 'none':
        df_result.loc[5:, 'Breed'] = 'other'
        graph_title = "Breed Distribution"
    else:
        df_result.loc[8:, 'Breed'] = 'other'
        graph_title = f'Breed Distribution for {dropdown_val} Rescue'
    # Create pie chart
    fig = px.pie(df_result, values='Total', names='Breed', hole=.3, 
                 title=graph_title)
    
    return dcc.Graph(figure=fig)


# This callback updates the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")])

def update_map(viewData, selected_row):

    dff = pd.DataFrame.from_dict(viewData)
    if selected_row:
        row = selected_row[0]
    else:
        row = 0

    return [
        dl.Map( center=[dff.iloc[row,13], dff.iloc[row, 14]], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9]),
                    html.P("Latitude: {}".format(dff.iloc[row, 13])),
                    html.P("Longitude: {}".format(dff.iloc[row, 14]))
                ])
            ])
        ])
    ]
    
    

app